# Energy Load Forecasting and Critical Scenario Simulation
**Author**: Mirza Shaheen Iqubal

This notebook combines:
- A time-series load forecasting model using LSTM
- A climate-energy scenario simulation

Both components directly support the goals of the *Siemens Energy Working Student (Data Analysis & Simulation)* role by evaluating historical and future grid load patterns influenced by climate data.

## 📥 Load and Weather Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Load datasets
energy = pd.read_csv('/mnt/data/energy_dataset.csv')
weather = pd.read_csv('/mnt/data/weather_features.csv')

# Parse datetime
energy['time'] = pd.to_datetime(energy['time'], utc=True)
weather['dt_iso'] = pd.to_datetime(weather['dt_iso'], utc=True)

# Merge datasets on datetime
df = pd.merge(energy, weather, left_on='time', right_on='dt_iso')
df = df[['time', 'total load actual', 'temp', 'humidity']].dropna()


## 🔍 Climate-Load Correlation Analysis

In [ ]:
sns.heatmap(df[['total load actual', 'temp', 'humidity']].corr(), annot=True)
plt.title('Correlation Matrix')
plt.show()

## ⚠️ Scenario Simulation: Coincidence of Peak Load and Extreme Temperature

In [ ]:
# Define thresholds
temp_thresh = df['temp'].quantile(0.90)
load_thresh = df['total load actual'].quantile(0.90)

df['high_temp'] = df['temp'] >= temp_thresh
df['high_load'] = df['total load actual'] >= load_thresh
df['critical_condition'] = df['high_temp'] & df['high_load']

df['month'] = df['time'].dt.month
monthly_counts = df.groupby('month')['critical_condition'].sum()
monthly_counts.plot(kind='bar', title='Monthly Frequency of Critical Conditions')
plt.ylabel('Count')
plt.xlabel('Month')
plt.tight_layout()
plt.show()

## 🤖 LSTM Model: Forecasting Energy Load

In [ ]:
# Use only one column for load
load_series = df.set_index('time')['total load actual'].resample('H').mean().fillna(method='ffill')

# Normalize data
scaler = MinMaxScaler()
scaled_load = scaler.fit_transform(load_series.values.reshape(-1, 1))

# Create sequences
seq_len = 24
X, y = [], []
for i in range(len(scaled_load) - seq_len):
    X.append(scaled_load[i:i+seq_len])
    y.append(scaled_load[i+seq_len])

X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(seq_len, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train
model.fit(X, y, epochs=3, batch_size=64, verbose=1)

## ✅ Job Description Alignment Summary
- Analyzed weather & load data over time ✔️
- Modeled critical grid stress conditions ✔️
- Built LSTM model for forecasting future load ✔️
- All tasks align with Siemens Energy’s simulation and data analysis objectives for grid design and efficiency ✔️